In [8]:
# Config Variables

# Define UCI Datasets to pull
# https://archive.ics.uci.edu/ml/datasets.php
uci_datasets = [1,9,10,16,60,87,89,92,162,165,186,189,211,242,244,270,275,291,294,300,320,332,368,374,381,390,409,464,471,477,492,519,544,547,560,563,565,597,601,713,849,851,857,880,890,925,942]
#183,

# List of file names provided in the R code from 2018 Half Ridge Paper
paper2018_datasets = [
    "house.world", "mortality", "cit.world", "prf.world", "bodyfat.world", 
    "car.world", "cloud", "dropout", "fat.world", "fuel.world", "glps",
    "homeless.world", "landrent.world", "mammal.world", "oxidants",
    "attractiveness.men", "attractiveness.women", "fish.fertility", 
    "oxygen", "ozone"
]

# Use label encoding or one hot encoding for categorical variables? values: 'label', 'onehot', 'none'
categorical_encoding = 'onehot'

# How to handle missing data? values: 'mean', 'median', 'mode', 'drop', 'zero', 'ffill', 'bfill', 'interpolate'
missing_data_handling = 'mean'

# Use all data (vs just training data) for OLS to determine co-efficient sign
ols_use_all_data = True

# Training set sizes to use (only training set sizes < 60% of total dataset size will be used)
#training_set_sizes = [10, 40, 160, 640, 2560, 10240, 40960]
training_set_sizes = [10, 40, 160, 610, 2560, 10240, 40960]
chain_lengths = [20000, 20000, 100000, 100000, 100000, 100000, 100000]

# Number of etas (prior standard deviation) to test between 1e-04 and 1e05 (recommend 20)
# Limits at 0 and infinity are calculated separately
# Note: In practice, sigma^2 (likelihood variance) is set to 1/eta^2 and
#       lambda (penalty) is set to 1/eta^4 
num_etas = 20

# Number of folds for cross-validation (set to 300 in 2018 paper, recommend testing with 20)
cv_folds = 20

# Overwrite model if it's already saved to file
overwrite_model = False

# Debugging Comments
verbose = False

In [2]:
# Load necessary libraries and helper functions

# import half_ridge functions from helpers/half_ridge.py
from helpers.half_ridge import *

# import testing functions from helpers/testing_functions.py
from helpers.testing_functions import *

# import dataset class
from helpers.model import Model

# import os and pickle
import os
import pickle
 
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

import time

In [9]:
# Load the UCI dataset, normalize the data, and split into training, validation, and test sets

# Loop over each dataset, loaded from file and formatted
for dataset in uci_datasets:
    model = Model()
    target = 0

    # iterate over the target (while the file exists)
    while os.path.isfile('uci_ml_datasets/ucirepo_' + str(dataset) + '_' + str(target) + '.csv'):

        model.import_from_file('uci_ml_datasets/ucirepo_' + str(dataset) + '_' + str(target) + '.csv')
        model.format_data(categorical_encoding, missing_data_handling)
        training_set_values = model.set_up_training_values(training_set_sizes, .6)


        # Check if a file exists for the dataset in the model_results folder
        if os.path.isfile(f"model_results/ucirepo_{dataset}_{target}.csv") and not overwrite_model:
            print(f"Model already exists for dataset {dataset}. Skipping...")
            # increment the target
            target += 1
            continue

        # Print the training set values for this dataset
        #print(f"Training Set Values: {training_set_values}")
        
        # Initialize the comparison results
        binary_comparisons = {}

        # Determine the signs of the coefficients using OLS (all data)
        if ols_use_all_data:
            ols_coefficients = model.get_ols_coefficients(True)
            weight_signs = np.sign(list(ols_coefficients.values()))

        # loop over the training set values
        for training_set_index, training_set_value in enumerate(training_set_values):

            # Timing
            set_start_time = time.time()
            print(f"Training Set Value: {training_set_value} for dataset {dataset}")

            # Generate cross-validation indices
            cv = model.cv_indexing(cv_folds, training_set_value)

            # Initialize the comparison results for this training set value
            binary_comparisons[training_set_value] = {}

            # Vary eta values from 10^-4 to 10^5, including 0 and inf, and loop over them
            etas = np.logspace(-4, 5, num=num_etas)
            etas = np.insert(etas, 0, 0)
            etas = np.append(etas, np.inf)
            for eta in etas:

                # Initialize the comparison results for this eta
                binary_comparisons[training_set_value][eta] = []

                # Crossvalidate over the folds for a specific eta
                for i in range(cv_folds):

                    # Print info about this iteration
                    if verbose: print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                    # Get training and test indices for the i-th fold
                    train_indices = np.where(cv[i] == 1)[0]
                    test_indices = np.where(cv[i] == 0)[0]
                    
                    # Split the data
                    X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                    y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                    # Get OLS coefficients for the training data
                    ols_coefficients = model.get_ols_coefficients(False, X_train, y_train)

                    # Determine the signs of the coefficients using OLS on only the training data
                    if not ols_use_all_data:
                        weight_signs = np.sign(list(ols_coefficients.values()))

                    '''
                    # Run the half ridge regression with mcmc
                    half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                    var_dict = {}
                    for variable in half_ridge_trace.posterior:
                        var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                    # Results into a dataframe
                    var_weights = pd.DataFrame(var_dict)
                    # Means for all the weights
                    var_means_array = var_weights.mean(axis=0)
                    # Convert the dataframe var_means_array to a dictionary with the column names as keys
                    half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                    '''
                    
                    # Run the half ridge regression with rejection sampling
                    start_time = time.time()
                    half_ridge_coefficients = half_ridge_rejection_sampling(weight_signs, ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                    end_time = time.time()
                    duration = end_time - start_time
                    if verbose: print(f"      Half ridge regression rejection sampling took {duration} seconds")

                    # Calculate the binary comparisons for the models
                    sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                    sorted_half_ridge_coefficients['intercept'] = 0
                    if verbose: print(f"      half-ridge coefficients: {sorted_half_ridge_coefficients}")
                    binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                    binary_comparisons[training_set_value][eta].append(binary_comparison)
                    if verbose: print(f"      binary comparison: {binary_comparison}")

                    '''
                    # Calculate SSE for the half ridge coefficients
                    half_ridge_sse = calculate_sse(sorted_half_ridge_coefficients, X_test, y_test)
                    #binary_comparisons[training_set_value][eta].append(half_ridge_sse)
                    print(f"half ridge sse: {half_ridge_sse}")
                    '''

            set_end_time = time.time()
            set_duration = set_end_time - set_start_time
            print(f"   Training Set {training_set_value} full run took {set_duration} seconds")


        
        # Save the binary comparisons to a file
        with open('model_results/ucirepo_' + str(dataset) + '_' + str(target) + '.csv', 'wb') as file:
            pickle.dump(binary_comparisons, file)
        
        # increment the target
        target += 1


Model already exists for dataset 1. Skipping...
Model already exists for dataset 9. Skipping...
Model already exists for dataset 10. Skipping...
Model already exists for dataset 16. Skipping...
Model already exists for dataset 60. Skipping...
Model already exists for dataset 87. Skipping...
Model already exists for dataset 89. Skipping...
Model already exists for dataset 89. Skipping...
Model already exists for dataset 89. Skipping...
Model already exists for dataset 92. Skipping...
Model already exists for dataset 162. Skipping...
Model already exists for dataset 165. Skipping...
Training Set Value: 10 for dataset 186
   Training Set 10 full run took 1.1920928955078125e-05 seconds
Training Set Value: 40 for dataset 186


In [6]:
# Load the 2018 Paper datasets, normalize the data, and split into training, validation, and test sets

training_set_sizes = [10, 20, 115] # Used for 2018 paper

# Loop over each dataset, loaded from file and formatted
for dataset in paper2018_datasets:
    model = Model()
    target = 0

    # iterate over the target (while the file exists)
    while os.path.isfile('2018paper_datasets/2018paper_' + str(dataset) + '_' + str(target) + '.csv'):

        model.import_from_file('2018paper_datasets/2018paper_' + str(dataset) + '_' + str(target) + '.csv')
        model.format_data(categorical_encoding, missing_data_handling)
        model.create_paired_data()
        training_set_values = model.set_up_training_values(training_set_sizes, .6)


        # Check if a file exists for the dataset in the model_results folder
        if os.path.isfile(f"model_results/2018paper_{dataset}_{target}.csv") and not overwrite_model:
            print(f"Model already exists for dataset {dataset}. Skipping...")
            # increment the target
            target += 1
            continue

        # Print the training set values for this dataset
        #print(f"Training Set Values: {training_set_values}")
        
        # Initialize the comparison results
        binary_comparisons = {}

        # Determine the signs of the coefficients using OLS (all data)
        if ols_use_all_data:
            ols_coefficients = model.get_ols_coefficients(True)
            weight_signs = np.sign(list(ols_coefficients.values()))

        # loop over the training set values
        for training_set_index, training_set_value in enumerate(training_set_values):

            # Timing
            set_start_time = time.time()
            print(f"Training Set Value: {training_set_value} for dataset {dataset}")

            # Generate cross-validation indices
            cv = model.cv_indexing(cv_folds, training_set_value)

            # Initialize the comparison results for this training set value
            binary_comparisons[training_set_value] = {}

            # Vary eta values from 10^-4 to 10^5, including 0 and inf, and loop over them
            etas = np.logspace(-4, 5, num=num_etas)
            etas = np.insert(etas, 0, 0)
            etas = np.append(etas, np.inf)
            for eta in etas:

                # Initialize the comparison results for this eta
                binary_comparisons[training_set_value][eta] = []

                # Crossvalidate over the folds for a specific eta
                for i in range(cv_folds):

                    # Print info about this iteration
                    if verbose: print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                    # Get training and test indices for the i-th fold
                    train_indices = np.where(cv[i] == 1)[0]
                    test_indices = np.where(cv[i] == 0)[0]
                    
                    # Split the data
                    X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                    y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                    # Get OLS coefficients for the training data
                    ols_coefficients = model.get_ols_coefficients(False, X_train, y_train)

                    # Determine the signs of the coefficients using OLS on only the training data
                    if not ols_use_all_data:
                        weight_signs = np.sign(list(ols_coefficients.values()))

                    '''
                    # Run the half ridge regression with mcmc
                    half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                    var_dict = {}
                    for variable in half_ridge_trace.posterior:
                        var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                    # Results into a dataframe
                    var_weights = pd.DataFrame(var_dict)
                    # Means for all the weights
                    var_means_array = var_weights.mean(axis=0)
                    # Convert the dataframe var_means_array to a dictionary with the column names as keys
                    half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                    '''
                    
                    # Run the half ridge regression with rejection sampling
                    start_time = time.time()
                    half_ridge_coefficients = half_ridge_rejection_sampling(weight_signs, ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                    end_time = time.time()
                    duration = end_time - start_time
                    if verbose: print(f"      Half ridge regression rejection sampling took {duration} seconds")

                    # Calculate the binary comparisons for the models
                    sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                    sorted_half_ridge_coefficients['intercept'] = 0
                    if verbose: print(f"      half-ridge coefficients: {sorted_half_ridge_coefficients}")
                    #binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                    binary_comparison = calculate_accuracy(sorted_half_ridge_coefficients, X_test, y_test, True)
                    binary_comparisons[training_set_value][eta].append(binary_comparison)
                    if verbose: print(f"      binary comparison: {binary_comparison}")

                    '''
                    # Calculate SSE for the half ridge coefficients
                    half_ridge_sse = calculate_sse(sorted_half_ridge_coefficients, X_test, y_test)
                    #binary_comparisons[training_set_value][eta].append(half_ridge_sse)
                    print(f"half ridge sse: {half_ridge_sse}")
                    '''

            set_end_time = time.time()
            set_duration = end_time - start_time
            print(f"   Training Set {training_set_value} full run took {set_duration} seconds")


        
        # Save the binary comparisons to a file
        with open('model_results/2018paper_' + str(dataset) + '_' + str(target) + '.csv', 'wb') as file:
            pickle.dump(binary_comparisons, file)
        
        # increment the target
        target += 1


Model already exists for dataset house.world. Skipping...
Model already exists for dataset mortality. Skipping...
Model already exists for dataset cit.world. Skipping...
Model already exists for dataset prf.world. Skipping...
Model already exists for dataset bodyfat.world. Skipping...
Model already exists for dataset car.world. Skipping...
Training Set Value: 10 for dataset cloud
   Training Set 10 full run took 7.152557373046875e-06 seconds
Training Set Value: 20 for dataset cloud
   Training Set 20 full run took 1.0013580322265625e-05 seconds
Training Set Value: 115 for dataset cloud
   Training Set 115 full run took 6.9141387939453125e-06 seconds
Training Set Value: 10 for dataset dropout
   Training Set 10 full run took 1.8358230590820312e-05 seconds
Training Set Value: 20 for dataset dropout
   Training Set 20 full run took 1.71661376953125e-05 seconds
Training Set Value: 115 for dataset dropout
   Training Set 115 full run took 1.811981201171875e-05 seconds
Training Set Value: 10